# Correspondencia en direcciones postales

In [1]:
import random
import string
import numpy as np
import tensorflow as tf

In [2]:
n = 10
street_names = ["diagon", "elm", "abbey", "gran", "python"]
street_type = ["callejon", "calle", "carrera", "via", "avenida"]
street_zips = [random.randint(20000, 29999) for i in range(5)]
numbers = [random.randint(1,999) for i in range(n)]

In [3]:
streets = [random.choice(street_names) for i in range(n)]
street_prefs = [random.choice(street_type) for i in range(n)]
zips = [random.choice(street_zips) for i in range(n)]
full_streets = [x + " " + y + " " + str(z) for x,y,z in zip(street_prefs, streets, numbers)]
reference_data = [list(x) for x in zip(full_streets, zips)]

In [4]:
reference_data

[['callejon gran 953', 24190],
 ['calle gran 905', 25215],
 ['via gran 617', 27127],
 ['callejon gran 177', 27127],
 ['callejon diagon 72', 27127],
 ['avenida abbey 938', 27889],
 ['calle gran 838', 22567],
 ['callejon gran 115', 24190],
 ['avenida abbey 980', 22567],
 ['via abbey 576', 22567]]

In [5]:
#Funcion que agrega errores a las direcciones

def create_typo(s, prob=0.75):
    if random.uniform(0,1)<0.75:
        rand_idx = random.choice(range(len(s)))
        s_list = list(s)
        s_list[rand_idx] = random.choice(string.ascii_lowercase)
        s = ''.join(s_list)
    return s

In [6]:
typo_streets = [create_typo(x) for x in streets]

In [7]:
typo_full_streets = [x+" "+y+" "+str(z) for x,y,z in zip(street_prefs, typo_streets, numbers)]
test_data = [list(x) for x in zip(typo_full_streets, zips)]

In [8]:
test_data

[['callejon gran 953', 24190],
 ['calle xran 905', 25215],
 ['via gron 617', 27127],
 ['callejon gran 177', 27127],
 ['callejon dirgon 72', 27127],
 ['avenida abbey 938', 27889],
 ['calle uran 838', 22567],
 ['callejon gran 115', 24190],
 ['avenida aubey 980', 22567],
 ['via ubbey 576', 22567]]

In [9]:
session = tf.Session()

In [10]:
test_address = tf.sparse_placeholder(dtype = tf.string)
test_zip = tf.placeholder(shape = [None, 1], dtype=tf.float32)

ref_address = tf.sparse_placeholder(dtype = tf.string)
ref_zip = tf.placeholder(shape=[None, n], dtype=tf.float32 )

In [11]:
zip_dist = tf.square(tf.sub(ref_zip, test_zip)) #distancia de numeros enteros
address_dist = tf.edit_distance(test_address, ref_address, normalize=True) #distancia de texto

### Distancia=0 cuando los objetos son iguales

### Similaridad=1 cuando los objetos son iguales

- $S(x,y) = 0$ si $x$ e $y$ son totalmente diferentes (no se parecen en nada)
- $S(x,x) = 1$, ya que todo objeto es similar (si no igual) a si mismo.
- $S(x,y) = \frac{D - d(x,y)}{D-d}$ donde $D$ es la mayor distancia entre dos objetos posibles, y $d$ es la menor.

In [12]:
zip_max = tf.gather(tf.squeeze(zip_dist), tf.argmax(zip_dist, 1)) #Distancia maxima
zip_min = tf.gather(tf.squeeze(zip_dist), tf.argmin(zip_dist, 1)) #Distancia minima
zip_sim = tf.div(tf.sub(zip_max, zip_dist), tf.sub(zip_max, zip_min))

**En texto la maxima distancia ya es 1 y la minima 0**

In [14]:
address_sim = tf.sub(1.0, address_dist)

### Combinacion lineal varicentrica

$$S(x,y) = \sum_{i=1}^k w_iS_k(x,y):\quad \sum_{i=1}^kw_i = 1$$

In [17]:
address_wi = 0.5
zip_wi = 1.0 -address_wi

weighted_sim = tf.add(tf.transpose(tf.mul(address_wi, address_sim)),tf.mul(zip_wi, zip_sim))

In [18]:
top_match_idx = tf.argmax(weighted_sim, 1) #direccion mas similar

In [19]:
def sparse_from_word_vector(word_vector):
    num_words = len(word_vector)
    idx = [[xi, 0, yi] for xi, x in enumerate(word_vector) for yi, y in enumerate(x)]
    chars = list(''.join(word_vector))
    return tf.SparseTensorValue(idx, chars, [num_words,1,1])

In [20]:
reference_address = [x[0] for x in reference_data]
reference_zips = np.array([[x[1] for x in reference_data]])

In [21]:
sparse_ref_set = sparse_from_word_vector(reference_address) #se convierte la direccion de referencia a SparseTensor

In [41]:
for i in range(n):
    test_address_entry = test_data[i][0]
    test_zip_entry = [[test_data[i][1]]] #manera para que pueda realizar las operaciones c
    
    #recordar que para cada palabra como hipotesis hay una verdadera palabra
    test_address_rep = [test_address_entry]*n #se multiplica la direccion mala por el num de direcciones correctas 
    sparse_test_set = sparse_from_word_vector(test_address_rep) #se convierte a SparseTensor
    
    feed_dict = {test_address: sparse_test_set,
                test_zip: test_zip_entry, #los numeros no necesitan convertirte a Sparse
                ref_address: sparse_ref_set,
                ref_zip: reference_zips} #los numeros no necesitan convertirte a Sparse
    
    best_match = session.run(top_match_idx, feed_dict=feed_dict)
    best_address = reference_address[best_match]
    [best_zip] = reference_zips[0][best_match]
    [[test_zip_aux]] = test_zip_entry
    
    print(best_match)
    print("Dirección original: "+str(test_address_entry)+ ", "+str(test_zip_aux))
    print("Dirección corregida: "+str(best_address)+", "+str(best_zip)+"\n")

[0]
Dirección original: callejon gran 953, 24190
Dirección corregida: callejon gran 953, 24190

[1]
Dirección original: calle xran 905, 25215
Dirección corregida: calle gran 905, 25215

[2]
Dirección original: via gron 617, 27127
Dirección corregida: via gran 617, 27127

[3]
Dirección original: callejon gran 177, 27127
Dirección corregida: callejon gran 177, 27127

[4]
Dirección original: callejon dirgon 72, 27127
Dirección corregida: callejon diagon 72, 27127

[5]
Dirección original: avenida abbey 938, 27889
Dirección corregida: avenida abbey 938, 27889

[6]
Dirección original: calle uran 838, 22567
Dirección corregida: calle gran 838, 22567

[7]
Dirección original: callejon gran 115, 24190
Dirección corregida: callejon gran 115, 24190

[8]
Dirección original: avenida aubey 980, 22567
Dirección corregida: avenida abbey 980, 22567

[9]
Dirección original: via ubbey 576, 22567
Dirección corregida: via abbey 576, 22567



/home/jess/anaconda3/envs/py34/lib/python3.4/site-packages/ipykernel/__main__.py:15: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
